In [1]:
# Sage maker notebook. This notebook uses the BileRental data and XGBoost algorithm
# The output file will be available for upload to kaggle for the kaggle score ( Need to add this to Readme)

In [4]:
# Install xgboost on the notebook instance
!conda install -y -c conda-forge xgboost

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs: 
    - xgboost


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    mkl_fft-1.0.10             |           py36_0         650 KB  conda-forge
    mkl_random-1.0.2           |           py36_0         1.3 MB  conda-forge
    py-xgboost-0.90            |           py36_4          73 KB  conda-forge
    tbb-2019.9                 |       hc9558a2_0         1.4 MB  conda-forge
    libxgboost-0.90            |       he1b5a44_4         2.4 MB  conda-forge
    openssl-1.0.2t             |       h14c3975_0    

In [6]:
# import numpy, pandas, sagemaker, xgboost

import sys
import math
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sagemaker
from sagemaker import get_execution_role

# XGBoost
import xgboost as xgb

In [7]:
# Sagemaker requires that the target variable be the first variable. 
# Define the columns to include count as the first vavariable
columns = ['count', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'year', 'month', 'day', 'dayofweek','hour']

In [10]:
df = pd.read_csv('train.csv', parse_dates=['datetime'], index_col=0)

In [12]:
df_test = pd.read_csv('test.csv', parse_dates=['datetime'], index_col=0)

In [13]:
# from the index feature get the year, month, day, dayofweek, and hour as new columns
def add_features(df):
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['dayofweek'] = df.index.dayofweek
    df['hour'] = df.index.hour

In [14]:
# add the new features to the df and df_test data frame
add_features(df)
add_features(df_test)

In [17]:
# for the train data frame look at the target column count 
df['count'].describe()

count    10886.000000
mean         4.591364
std          1.419454
min          0.693147
25%          3.761200
50%          4.983607
75%          5.652489
max          6.885510
Name: count, dtype: float64

In [16]:
# Log on the count column
df['count'] = df['count'].map(np.log1p)

In [18]:
# copy the data in the data frame into a csv file. The csv file will be uploaded to s3 bucket to be used by the model
df.to_csv('bike_all.csv', index=True, index_label='datetime', columns=columns)